# Evaluation and metrics

In [2]:
import keras
import sys
sys.path.append('../')
from datasets import CD_Dataset
from models import Unet
from utility import show_batches, from_categorical
import numpy as np
from datasets import combine_y_w
import keras
import time

seed = int((time.time()*1e6)%1e6)
np.random.seed(seed)
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)

dataset_path = '../CD_Dataset'
trained_models_path = '../trained_models'

input_patch_size = [350,350]
input_channels = [3]
output_channels = [2]
model_input_size = input_patch_size + input_channels

dataset = CD_Dataset( path=dataset_path, train_y_path="train_y",  eval_y_path="eval_y",  fit=True, download=True, num_classes=output_channels[0] )

means = dataset.mean_features()
stds = dataset.std_features()

Using TensorFlow backend.


('mean_features: ', array([ 0.86503507,  0.83272585,  0.72929356]))
('std_features: ', array([ 0.01255889,  0.01785996,  0.05625756]))


In [ ]:
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion

def Pc(Y,Y_hat,tetha=5):
    Bgt = Y - binary_erosion(Y,structure=np.ones((3,3)))
    Bps = Y_hat - binary_erosion(Y_hat,structure=np.ones((3,3)))
    D = distance_transform_edt(Bgt)
    D_Bpd = D[Bps==1.0]
    Nominator = len(D_Bpd[D_Bpd>tetha].flatten())
    Denominator = len(Bgt[Bgt==1.0].flatten())
    return Nominator/float(Denominator)
    